In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
!pip install torchvision
!pip install seaborn
!pip install tqdm
!pip install torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import make_grid
from tqdm.notebook import tqdm
import random
import warnings
from PIL import Image
warnings.filterwarnings('ignore')

In [ ]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

data_path = '/kaggle/input/crop-disease-detection-dataset/Plant Village Dataset/'
train_folder = os.path.join(data_path, 'Train')
test_folder = os.path.join(data_path, 'Test')
val_folder = os.path.join(data_path,'Val')

classes = os.listdir(train_folder)
unique_plant_diseases = []
for item in classes:
    disease = item.split('_')[0]
    if disease not in unique_plant_diseases:
        unique_plant_diseases.append(disease)
print("Number of unique plant diseases:", len(unique_plant_diseases))
print("Plants:", unique_plant_diseases)

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG16 expects 224x224 images
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

In [ ]:
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_ds = ImageFolder(train_folder, transform=train_transform)
test_ds = ImageFolder(test_folder, transform=eval_transform)
val_ds = ImageFolder(val_folder, transform=eval_transform)

print("Number of training images:", len(train_ds))
print("Number of test images:", len(test_ds))
print("Number of validation images:", len(val_ds))
print("Number of classes:", len(train_ds.classes))
print("Classes:", train_ds.classes)

img, label = train_ds[4587]
print("Image shape:", img.shape)
print("Label:", label, train_ds.classes[label])

In [ ]:
# Function to denormalize images for visualization
def denormalize(tensor):
    mean = torch.tensor([0.485, 0.456, 0.406]).reshape(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).reshape(3, 1, 1)
    return tensor * std + mean

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.imshow(denormalize(img).permute(1, 2, 0).clamp(0, 1))
ax1.set_title("Original Image")
ax2.imshow(1 - denormalize(img).permute(1, 2, 0).clamp(0, 1))
ax2.set_title("Inverted Image")
plt.show()

train_data_overview = DataLoader(train_ds, batch_size=32, shuffle=True)
images, labels = next(iter(train_data_overview))
sample_indices = random.sample(range(len(images)), 6)
plt.figure(figsize=(20, 4))
for i, idx in enumerate(sample_indices):
    plt.subplot(1, 6, i+1)
    plt.imshow(denormalize(images[idx]).permute(1, 2, 0).clamp(0, 1))
    plt.title(f"{train_ds.classes[labels[idx]]}")
    plt.axis("off")
plt.suptitle("Dataset Overview\n\n\n\n\n\n\n\n\n\n\n\n\n6 Random Training Images.", fontsize=15)
plt.show()

In [ ]:
# Increased batch size for faster training and better gradient estimates
batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

for imgs, lbls in train_loader:
    plt.figure(figsize=(20, 8))
    plt.imshow(make_grid(denormalize(imgs), nrow=16).permute(1, 2, 0).clamp(0, 1))
    plt.axis('off')
    break

diseases = os.listdir(train_folder)
class_counts = {cls: len(os.listdir(os.path.join(train_folder, cls))) for cls in diseases}
df = pd.DataFrame(list(class_counts.items()), columns=["Disease Class", "Number of Images"])
plt.figure(figsize=(15,5))
sns.barplot(data=df, x='Disease Class', y='Number of Images')
plt.xticks(rotation=90)
plt.title("Dataset Distribution\nNumber of Images per Disease Class.")
plt.show()

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss': loss, 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_loss = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_acc = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print(f"Epoch [{epoch}], train_loss: {result['train_loss']:.4f}, val_loss: {result['val_loss']:.4f}, val_acc: {result['val_acc']:.4f}")

class Plant_Disease_VGG16(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Load pre-trained VGG16 model
        self.network = models.vgg16(pretrained=True)
        
        # Freeze more layers for fine-tuning
        for param in list(self.network.features.parameters())[:-8]:  # Freeze more early layers
            param.requires_grad = False
            # Get the number of input features for the classifier
        num_ftrs = self.network.classifier[-1].in_features
        
        # Replace the classifier with our custom classifier
        # Added an additional layer for better feature representation
        self.network.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),  # Added layer (original VGG16 has 4096->4096->1000)
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(1024, 38)  # Output layer for 38 classes
        )
        
        # Initialize the weights of the new layers
        for m in self.network.classifier.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, xb):
        return self.network(xb)

def get_default_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)

device = get_default_device()
print("Using device:", device)

vgg_model_ft = Plant_Disease_VGG16()
print(vgg_model_ft)  # Check model before moving to device
vgg_model_ft = to_device(vgg_model_ft, device)
print(vgg_model_ft)  # Check if it moves successfully

train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    # Using NAdam with improved parameters
    optimizer = opt_func(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-5)
    
    # Learning rate scheduler for better convergence
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    best_val_acc = 0.0
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            optimizer.zero_grad()
        
        torch.cuda.empty_cache()
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        
        # Step the scheduler based on validation loss
        scheduler.step(result['val_loss'])
        
        # Save the best model based on validation accuracy
        if result['val_acc'] > best_val_acc:
            best_val_acc = result['val_acc']
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"Model saved with val_acc: {best_val_acc:.4f}")
    
    # Load the best model before returning
    model.load_state_dict(torch.load('best_model.pth'))
    return history

# Increased epochs and adjusted learning rate for better convergence
history_vgg_ft = fit(40, 0.0001, vgg_model_ft, train_loader, val_loader, opt_func=torch.optim.NAdam)

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, '-bx', label="Train Loss")
    plt.plot(val_losses, '-rx', label="Val Loss")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss vs. Epochs')
    plt.grid(True)
    plt.show()

def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.figure(figsize=(10, 6))
    plt.plot(accuracies, '-x', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. Epochs')
    plt.grid(True)
    plt.show()

plot_accuracies(history_vgg_ft)
plot_losses(history_vgg_ft)

print("Evaluation on Test Set (Fine-tuned VGG16):")
print(evaluate(vgg_model_ft, test_loader))

# Generate confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def plot_confusion_matrix(model, test_loader, classes):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
    print(classification_report(all_labels))

In [ ]:
# Save the complete model
torch.save(vgg_model_ft, 'plant_disease_model_complete.pth')

# Save just the state dictionary (recommended, more portable)
torch.save(vgg_model_ft.state_dict(), 'plant_disease_model_state_dict.pth')

# Save model architecture and parameters as a checkpoint
"""checkpoint = {
    'model_state_dict': vgg_model_ft.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),  # assuming optimizer is still in scope
    'classes': train_ds.classes,
    'epoch': len(history_vgg_ft)
}
torch.save(checkpoint, 'plant_disease_model_checkpoint.pth')"""